In [1]:
import time
from importlib.resources import files
import numpy as np
from Consensus import *

In [2]:
INPUT_TREE_PATH = files("Consensus.sample10").joinpath(f"lowsample3.tre")
input_trees = TreeList_with_support.get(path = INPUT_TREE_PATH, schema = "newick")
majority = input_trees.majority_rule_consensus()

In [3]:
stdg = STDGreedyConsensus(input_trees)

Creating Bipartition List etc... time passed : 0.039608001708984375
TIME_number_of_independent_bipartitions:  31
TIME_Creating_DIST: 0.00
TIME_Creating_TD: 0.18


In [4]:
stdg.BipartitionP

array([3, 2, 3, 2, 4, 3, 2, 2, 2, 4, 4, 2, 2, 4, 5, 3, 4, 4, 4, 5, 4, 4,
       5, 4, 5, 2, 4, 3, 5, 2, 2])

In [5]:
stdg.DIST.shape

(32, 32)

In [6]:
stdg.n_bipartitions

31

In [11]:
over_bipartitions = np.zeros(stdg.n_bipartitions)
for i in range(stdg.n_bipartitions):
    over_count = 0
    for j in range(stdg.n_bipartitions):
        if(stdg.DIST[i,j] <= stdg.BipartitionP[i] - 1): over_count += 1
    over_bipartitions[i] = over_count/stdg.n_bipartitions

In [12]:
over_bipartitions

array([0.61290323, 0.41935484, 0.4516129 , 0.22580645, 0.96774194,
       0.64516129, 0.19354839, 0.22580645, 0.22580645, 0.93548387,
       0.83870968, 0.19354839, 0.19354839, 0.93548387, 1.        ,
       0.64516129, 0.96774194, 0.96774194, 0.96774194, 1.        ,
       0.93548387, 0.93548387, 1.        , 0.93548387, 1.        ,
       0.12903226, 0.96774194, 0.51612903, 1.        , 0.29032258,
       0.29032258])